In [ ]:
!unzip "/content/Bats300.zip"

# Imports

In [ ]:
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.pipeline import Pipeline

import io
import numpy as np
from PIL import Image


In [ ]:
from numpy import unique
from numpy import where
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.cluster import MeanShift

**Preprocessing (Dimension Reduction)**

In [ ]:


dataset_path = "/content/Bats300"

data = load_files(dataset_path, shuffle=True, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)


In [ ]:
X_train = [np.asarray(Image.open(io.BytesIO(img)).convert("L")).flatten() for img in X_train]
X_test = [np.asarray(Image.open(io.BytesIO(img)).convert("L")).flatten() for img in X_test]

 **K-Means Semi-Supervised Learning**

In [ ]:
#if you only have 50 labeled instances
n_labeled = 50

In [ ]:
kmeans = KMeans(n_clusters=50, random_state=42)
X_digits_dist = kmeans.fit_transform(X_train)

#find the representative images closest to each centroid
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
# X_representative_digits = X_train.tolist()[representative_digit_idx]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
X_train[0], representative_digit_idx

In [ ]:
# X_train[representative_digit_idx]
X_representative_digits = [X_train[idx] for idx in representative_digit_idx]

In [ ]:
plt.figure(figsize=(8, 2))
for index, X_representative_digit in enumerate(X_representative_digits):
    plt.subplot(50 // 10, 10, index + 1)
    plt.imshow(X_representative_digit.reshape(40, 40), cmap="binary", interpolation="bilinear")
    plt.axis('off')

plt.show()

In [ ]:
y_representative_digits = np.array([
    3, 0, 1, 1, 3, 1, 2, 2, 2, 2,
    1, 2, 2, 2, 2, 2, 0, 1, 1, 3,
    1, 3, 1, 1, 3, 2, 2, 1, 1, 2,
    3, 3, 3, 2, 2, 1, 2, 1, 2, 3,
    2, 3, 1, 2, 2, 2, 2, 3, 3, 2])

In [ ]:
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(X_test, y_test)

**Affinity Propogation**

In [ ]:


# define the model
model = AffinityPropagation(damping=0.7)

# train the model
model.fit(X_train)

# assign each data point to a cluster
result = model.predict(X_train)

# get all of the unique clusters
clusters = unique(result)

len(clusters)

In [ ]:
len(result), len(X_train)

In [ ]:
type(X_test)

In [ ]:
un = np.unique(result)
result, un

In [ ]:
arr=[]
for val in un:
  indices = np.where(result == val)

  X_test_np = np.asarray(X_train)

  idk_what_to_call_this = X_test_np[indices]

  plt.figure(figsize=(8, 2))
  for index, X_representative_digit in enumerate(idk_what_to_call_this):
      plt.subplot(50 // 10, 10, index + 1)
      plt.imshow(X_representative_digit.reshape(40, 40), cmap="binary", interpolation="bilinear")
      plt.axis('off')
  plt.show()
  arr.append(int(input()))

In [ ]:
X_test_np[0]

In [ ]:
accuracy = accuracy_score(y_test, result)
accuracy

# Applying Erosion Preprocessing and Pixel Value masking

Perhaps some issue's with overlapping classes may be resolved if the data is preprocessed and bat's edges are eroded.

In [ ]:
dataset_path = "/content/Bats300"

data = load_files(dataset_path, shuffle=True, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

In [ ]:
X_img_train = [np.asarray(Image.open(io.BytesIO(img)).convert("L")) for img in X_train]
X_img_test = [np.asarray(Image.open(io.BytesIO(img)).convert("L")) for img in X_test]

In [ ]:
import cv2
import numpy as np

# Read your image
image = X_img_test[133]

# Define a kernel for erosion
kernel = np.ones((3, 3), np.uint8)

# Perform erosion
eroded_image = cv2.erode(image, kernel, iterations=1)

pixel_mask = np.copy(eroded_image)
pixel_mask[pixel_mask>np.uint8(255/5)] = 255

# Display the original and eroded images
plt.subplot(2 // 1, 3, 1)
plt.imshow(image, cmap="binary")
plt.subplot(2 // 1, 3, 2)
plt.imshow(eroded_image, cmap="binary")
plt.subplot(2 // 1, 3, 3)
plt.imshow(pixel_mask, cmap="binary")

plt.show()


In [ ]:
pixel_mask

In [ ]:
eroded_image


## Applying Erosion to Whole Dataset
### Steps:
* Import Dataset
* Apply Erosion
* Flatten Image after Erosion
* Apply Affinity Propogation

X_img_train, X_img_test, y_train, y_test

In [ ]:
"""
Apply erosion to X_images and y_images
"""

def apply_erosion(arr, pixel_masking: bool=False):
  elements, dim1, dim2 = np.shape(arr)

  # New numpy array
  new_arr = np.zeros((elements, dim1*dim2))

  # Define a kernel for erosion
  kernel = np.ones((3, 3), np.uint8)


  for idx, img in enumerate(arr):
    # Perform erosion
    arr[idx] = cv2.erode(img, kernel, iterations=1).flatten()
    if pixel_masking:
      arr[idx][arr[idx]>np.uint8(255/5)] = 255
  return arr

X_images = apply_erosion(X_img_train, pixel_masking=True)
y_images = apply_erosion(X_img_test, pixel_masking=True)

#### Applying Affinity Propogation

In [ ]:
# define the model
model = AffinityPropagation(damping=0.7)

# train the model
model.fit(X_img_train)

# assign each data point to a cluster
result = model.predict(X_img_train)

# get all of the unique clusters
clusters = unique(result)

len(clusters)

### Affinity Propogation: Semi Supervised Learning with Erosion
**Steps:**
* Find Unique Clusters
* Label Unique Clusters
* Predict Test set with new Unique Cluster labels

In [ ]:
un = np.unique(result)
result, un

In [ ]:
arr=[]

def semi_super(arr: np.ndarray, result: np.ndarray, un: np.ndarray, label_pass: bool=True):
  np_arr = []
  for val in un:
    indices = np.where(result == val)
    cluster_images = np.asarray(arr)[indices]

    plt.figure(figsize=(8, 2))
    for index, img in enumerate(cluster_images):
        if index > 49:
          break
        plt.subplot(50 // 10, 10, index + 1)
        plt.imshow(img.reshape(40, 40), cmap="binary", interpolation="bilinear")
        plt.axis('off')
    plt.show()
    if label_pass:
      np_arr.append(int(input()))
  return np_arr

val = semi_super(X_img_train, result, un, label_pass=False)

In [ ]:
eroded_result = result
for un_ele, val_ele in zip(un, val):
  eroded_result[eroded_result == un_ele] = val_ele

In [ ]:
np.shape(eroded_result)

In [ ]:
np.shape(y_train)

In [ ]:
accuracy = accuracy_score(y_train, eroded_result)
accuracy